In [39]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [40]:
import pandas as pd
import numpy as np
import gensim
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [41]:
df = pd.read_csv('/content/training1600000.csv',encoding = "ISO-8859-1",header=None)

In [42]:

df.columns = ["polarity",'id','date','query','user','tweet']
df

,polarity,id,date,query,user,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [43]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+',' ',text)
    text = re.sub('[^a-zA-Z]',' ',text)
    text = word_tokenize(text)
    text = [item for item in text if item not in stop_words]
    text = [lemma.lemmatize(w) for w in text]
    text = [i for i in text if len(i)>2]
    text = ' '.join(text)
    return text

In [44]:
stop_words = set(stopwords.words('english'))
lemma = WordNetLemmatizer()

In [45]:
df_new = df.sample(n=100000)

In [46]:
df_new['polarity'].value_counts()

0    50069
4    49931
Name: polarity, dtype: int64

In [47]:
df_new['clean_tweet'] = df_new['tweet'].apply(clean_text)

In [48]:
x_train = df_new['clean_tweet']

In [49]:

tweets = df_new['clean_tweet'].tolist()
words = []
for i in range(len(tweets)):
  words.append(tweets[i].split())


In [53]:
import gensim.corpora as corpora
id2word = corpora.Dictionary(words)
texts = words
corpus = [id2word.doc2bow(text) for text in texts]

In [54]:
from pprint import pprint
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10)
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.022*"love" + 0.015*"haha" + 0.010*"lol" + 0.009*"video" + 0.008*"see" + '
  '0.007*"thats" + 0.007*"read" + 0.007*"song" + 0.006*"watching" + '
  '0.006*"thanks"'),
 (1,
  '0.014*"time" + 0.014*"need" + 0.008*"get" + 0.008*"got" + 0.006*"house" + '
  '0.006*"amp" + 0.006*"coffee" + 0.005*"way" + 0.005*"money" + 0.005*"long"'),
 (2,
  '0.033*"work" + 0.019*"sleep" + 0.019*"going" + 0.018*"night" + 0.015*"get" '
  '+ 0.012*"bed" + 0.009*"hour" + 0.009*"early" + 0.008*"bored" + '
  '0.008*"time"'),
 (3,
  '0.014*"know" + 0.012*"well" + 0.011*"think" + 0.009*"thank" + 0.009*"hurt" '
  '+ 0.008*"movie" + 0.008*"need" + 0.008*"away" + 0.007*"much" + '
  '0.007*"looking"'),
 (4,
  '0.040*"like" + 0.025*"feel" + 0.021*"really" + 0.015*"twitter" + '
  '0.012*"nice" + 0.011*"want" + 0.010*"dont" + 0.010*"wan" + 0.009*"know" + '
  '0.009*"better"'),
 (5,
  '0.019*"thanks" + 0.016*"tweet" + 0.013*"wish" + 0.012*"could" + 0.009*"got" '
  '+ 0.008*"phone" + 0.008*"nothing" + 0.007*"yesterd